In [2]:
try:
    import numpy as np
    import pandas as pd
    from dask_ml.preprocessing import StandardScaler
    import gc
    import time
    import dask.dataframe as dd
    from dask.distributed import Client, progress # see the sys load
    
except: ImportError(), 'Some modules have not loaded'

In [3]:
# set workers
client = Client(n_workers=2, threads_per_worker=2, memory_limit='3GB')
client

/home/volodymyr/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)


Client Scheduler: tcp://127.0.0.1:35867 Dashboard: http://127.0.0.1:33551/status,Cluster Workers: 2 Cores: 4 Memory: 6.00 GB


In [ ]:
# setting the number of rows for the CSV file
start_time = time.time()

N = 5_000_000
columns = 257

# create DF 
df = pd.DataFrame(np.random.randint(999, 999999, size=(N, columns)), columns=['level_%s' % i for i in range(0, columns)])

print('%s seconds' % (time.time() - start_time))

In [ ]:
# save df to csv 

start_time = time.time()

df.to_csv('random.csv', sep=',')

print('%s seconds' % (time.time() - start_time)) # 877.5422155857086 seconds, 8.9 G

In [ ]:
gib = 'random.csv' # 5 mln records! 

In [4]:
class LoadBigCsvFile_TRAIN:
    
    '''load data from tsv, transform, scale, add two columns'''
    
    def __init__(self, train, scaler=StandardScaler(copy=False)):

        self.train = train
        self.scaler = scaler
    
    def read_data(self):
        # use dask for read with C 
        try:
            data_train = dd.read_csv(self.train, sep=',', header=None, skiprows=1, \
                                     dtype={n:'int16' for n in range(1, 300)}, engine='c')            
        except: MemoryError, 'not enough memory'
        
        # fit from train and scale to test
        self.scaler.fit(data_train.iloc[:,1:])
        temp = self.scaler.transform(data_train.iloc[:,1:])
                
        # index of max element
        temp['max_feature_2_index'] = temp.idxmax(axis=1)
        
        # calculate absolute diviation from max value in row
        temp['max_feature_2_abs_mean_diff'] = abs(temp.max(axis=1) - temp.mean(axis=1))
        
        # set id columnsD
        temp['job_id'] = data_train.iloc[:,0] 
        
        del data_train # del not needed
        
        return temp

In [ ]:
'''init the class,'''

start_time = time.time()

rer = LoadBigCsvFile_TRAIN(gib, StandardScaler(copy=False)).read_data()

print('%s seconds' % (time.time() - start_time)) # 10 ces

In [ ]:
gc.collect()

In [ ]:
# save to HDF

start_time = time.time()

rer.to_hdf('random.hdf',  key='df1')

print('%s seconds' % (time.time() - start_time)) #152.77526926994324 seconds!!! 10.4 G

In [ ]:
# read from HDF

start_time = time.time()

hdf_read = dd.read_hdf('random.hdf', key='df1', mode='r', chunksize=10000)

print('%s seconds' % (time.time() - start_time)) # 0.03516435623168945 seconds!!! 

In [ ]:
hdf_read.head(3) # read head

In [ ]:
assert len(hdf_read) == 5_000_000 # test 5ml records